### 手柄操作
在此示例中，我们将使用连接到Web浏览器的游戏手柄远程控制Jetbot。

### 创建游戏手柄控制器
我们要做的第一件事是创建``Controller`组件的实例，我们将用它来驱动机器人。该``Controller``窗口组件需要一个``index``参数，该参数是标识控制器的数字。如果您连接了多个控制器，或者某些游戏手柄显示为多个控制器时会很重要。用来确定您正在使用的控制器的索引。

1. 访问http://html5gamepad.com
2. 在您使用的游戏板上按一些按钮
3. 记住响应按钮按下的游戏手柄的``index``

接下来，我们将使用该索引创建并显示我们的控制器。

In [1]:
import ipywidgets.widgets as widgets

controller = widgets.Controller(index=0)  # replace with index of your controller

display(controller)

Controller()

即使索引正确，您也可能会看到文本``Connect gamepad and press any button``。这是因为游戏手柄尚未连接本notebook。按下一个按钮，您应该看到游戏手柄组件出现在上方。

### 将游戏手柄控制器连接到机器人的马达
现在，即使我们已经连接了游戏手柄，但是尚未将控件附加到机器人上！我们要附加的第一个也是最简单的控件是电机控件。我们将使用``dlink``函数将其连接到左右垂直轴。与``link``函数不同，``dlink``函数不允许我们在``source``和``target``之间转换。由于控制器轴偏离了我们认为对电机控制的直觉，因此我们将使用一个*lambda*函数来对该值取反。

> 注意：如果你触碰游戏控制器，下一个单元格将会移动机器人！

In [2]:
from jetbot import Robot
import traitlets

#注意，可以对小车设置的速度范围为0到200
robot = Robot()
right_link = traitlets.dlink((controller.axes[5], 'value'), (robot.right_motor, 'value'), transform=lambda x: -x * 150)

# 初始化舵机

In [3]:
from jetbot import Pwm
direction = Pwm()
#direction.switch(direction.turn_middle)

# button 5 居中

In [4]:
def steering_direction_middle(change):
    direction.switch(direction.turn_middle)
controller.buttons[0].observe(steering_direction_middle)

# button 6 控制左转

In [5]:
def steering_direction_left(change):
    direction.switch(direction.turn_left3)
controller.buttons[3].observe(steering_direction_left)

太棒了！现在，我们的机器人应该可以对游戏手柄控制器的动作做出响应。现在我们想要从摄像机观看实时视频！

# button 7 控制右转

In [6]:
def steering_direction_right(change):
    direction.switch(direction.turn_right3)
controller.buttons[1].observe(steering_direction_right)

# 前进

In [7]:
def move_forward(change):
    robot.set_motors(10, 130)
    time.sleep(1)
    robot.stop()
controller.buttons[5].observe(move_forward)

# 后退

In [8]:
def move_backward(change):
    robot.set_motors(0, -130)
    time.sleep(1)
    robot.stop()
controller.buttons[7].observe(move_backward)

# 停止

In [9]:
def move_stop(change):
    robot.stop()
controller.buttons[6].observe(move_stop)

### 创建并显示图组件
首先，让我们显示一个``Image``组件，用它来显示实时摄像机的图像。我们设置``height``和``width``设置为300像素，这样就不会占用太多空间。

仅供参考：高度和宽度仅影响浏览器端的渲染，而不影响从机器人到浏览器的网络传输之前的原始图像分辨率。

In [10]:
image = widgets.Image(format='jpeg', width=300, height=300)

display(image)

Image(value=b'', format='jpeg', height='300', width='300')

### 创建相机实例
好吧，目前没有图像呈现，因为我们还没有设置值！我们可以通过创建我们的``Camera`` 类并将``value``的属性附加到image的value属性来实现。

首先，让我们创建相机实例，我们调用该``instance``方法，如果尚未创建新相机，它将创建一个新相机。如果已经存在，则此方法将返回现有相机。

In [11]:
from jetbot import Camera

camera = Camera.instance(width=224, height=224, fps=10)

### 将相机连接到图像组件

我们的相机类当前仅产生BGR8（蓝色，绿色，红色，8位）格式的值，而我们的图像组件则接受压缩的*JPEG*值。要将相机连接到图像，我们需要插入``bgr8_to_jpeg``函数到链接中做转换。我们像下面这样做

In [12]:
from jetbot import bgr8_to_jpeg

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

现在，您应该看到上面显示的实时视频！

> 提醒：您可以右键单击单元格，然后选择``Create New View for Output``在单独的窗口中显示该单元格。

### 使用游戏手柄按钮保存快照
现在，我们希望能够从机器人中保存一些图像。让我们做一下，手柄按钮[4]来保存当前实时图像的快照。我们会将图像保存在``snapshots/``目录中，并使用``uuid``包确保其名称唯一。我们使用``uuid1``标识符，因为它还会对日期和MAC地址进行编码，以备后面使用。

In [14]:
import uuid
import subprocess
import numpy as np
import cv2

subprocess.call(['mkdir', '-p', 'snapshots'])

snapshot_image = widgets.Image(format='jpeg', width=300, height=300)

def save_snapshot(change):
    # save snapshot when button is pressed down
    if change['new']:
        file_path = 'snapshots/' + str(uuid.uuid1()) + '.jpg'
        
        # write snapshot to file (we use image value instead of camera because it's already in JPEG format)
        with open(file_path, 'wb') as f:
            f.write(image.value)
            
        # display snapshot that was saved
        snapshot_image.value = image.value


controller.buttons[4].observe(save_snapshot, names='value')

car_widget = widgets.Image(format='jpeg', width=300, height=300)

def my_car(camera_image):
    image0 = np.copy(camera_image)
    image0 = cv2.line(image0, (100,224), (24,200), (0,255,255), 3)
    image0 = cv2.line(image0, (124,224), (200,200), (0,255,255), 3)
    image0 = cv2.line(image0, (106,224), (106,212), (0,255,0), 3)
    image0 = cv2.line(image0, (118,224), (118,212), (0,255,0), 3)
    image0 = cv2.line(image0, (106,212), (118,212), (0,255,0), 3)
    jpeg_image = bgr8_to_jpeg(image0)
    return jpeg_image

traitlets.dlink((camera, 'value'), (car_widget, 'value'), transform=my_car)
display(widgets.HBox([car_widget]))

display(controller)

Controller(axes=(Axis(value=0.003921627998352051), Axis(value=0.003921627998352051), Axis(value=0.003921627998…

### 结论
本示例就是这样，玩得开心！